In [1]:
from collections import defaultdict
import heapq

subway_lines = {

    '1': ["소요산","동두천","보산","동두천중앙","지행","덕정","덕계","양주","녹양","가능",
           "의정부","회룡","망월사","도봉산","도봉","방학","창동","녹천","월계","성북",
           "석계","신이문","외대앞","회기","청량리","제기동","신설동","동묘앞","동대문","종로5가",
           "종로3가","종각","시청","서울역","남영","용산","노량진","대방","신길","영등포",
           "신도림","구로", "가산디지털단지","독산","금천구청","석수","관악","안양","명학",
           "금정","군포","당정","의왕","성균관대","화서","수원","세류","병점","세마","오산대","오산","진위","송탄","서정리","지제","평택","성환","직산","두정",
           "천안","봉명","쌍용","아산","배방","온양온천","신창"],
    '1인천':["구로","구일","개봉","오류동","온수","역곡","소사","부천","중동",
           "송내","부개","부평","백운","동암","간석","주안","도화","제물포","도원","동인천","인천"],

    '1서동탄': ["병점","서동탄"],

    '1광명' : ["금천구청","광명"],
    '2': ["시청","을지로입구","을지로3가","을지로4가","동대문역사문화공원","신당","상왕십리",
          "왕십리","한양대","뚝섬","성수","건대입구","구의","강변","잠실나루","잠실","신천",
          "종합운동장","삼성","선릉","역삼","강남","교대","서초","방배","사당","낙성대","서울대입구",
          "봉천","신림","신대방","구로디지털단지","대림","신도림","문래","영등포구청","당산","합정",
          "홍대입구","신촌","이대","아현","충정로","시청"],
    '2신설동' : ["성수","용답","신답","용두","신설동"],
    '2까치산' : ["신도림","도림천","양천구청","신정네거리","까치산"],
    '3': ["대화","주엽","정발산","마두","백석","대곡","화정","원당","삼송","지축",
          "구파발","연신내","불광","녹번","홍제","무악재","독립문","경복궁","안국","종로3가",
          "을지로3가","충무로","동대입구","약수","금호","옥수","압구정","신사","잠원",
          "고속터미널","교대","남부터미널","양재","매봉","도곡","대치","학여울","대청",
          "일원","수서","가락시장","경찰병원","오금"],
    '4': ["진접","오남","별내별가람","당고개","상계","노원","창동","쌍문","수유","미아",
          "미아삼거리","길음","성신여대입구","한성대입구","혜화","동대문","동대문역사문화공원",
          "충무로","명동","회현","서울역","숙대입구","삼각지","신용산","이촌","동작","이수",
          "사당","남태령","선바위","경마공원","대공원","과천","정부과천청사","인덕원","평촌",
          "범계","금정","산본","수리산","대야미","반월","상록수","한대앞","중앙","고잔","공단",
          "안산","신길온천","정왕","오이도"],
    '5하남검단산': ["방화","개화산","김포공항","송정","마곡","발산","우장산","화곡","까치산","신정",
          "목동","오목교","양평","영등포구청","영등포시장","신길","여의도","여의나루","마포",
          "공덕","애오개","충정로","서대문","광화문","종로3가","을지로4가","동대문역사문화공원",
          "청구","신금호","행당","왕십리","마장","답십리","장한평","군자","아차산","광나루",
          "천호","강동","길동","굽은다리","명일","고덕","상일동","강일","미사","하남풍산","하남시청","하남검단산"],
    '5마천':["강동","둔촌동","올림픽공원","방이",
          "오금","개롱","거여","마천"]
}

# 그래프 구조: 각 역 노드를 키로, 연결된 (노드, 가중치) 튜플의 리스트를 값으로 저장
graph = defaultdict(list)

# 각 역 이름이 어떤 노선들에 포함되어 있는지 저장
station_to_lines = defaultdict(list)

# 1. 모든 노선에 대해, 역들 간 인접 관계를 그래프에 추가하고
#    각 역이 포함된 노선 정보를 따로 저장
for line_key, stations in subway_lines.items():
    for i in range(len(stations) - 1):
        u = (line_key, stations[i])      # 현재 역
        v = (line_key, stations[i + 1])  # 다음 역

        # 양방향 간선 추가 (역 간 이동 시간: 2분)
        graph[u].append((v, 2))
        graph[v].append((u, 2))
        # 역을 노선에 포함
        station_to_lines[stations[i]].append(line_key)
    #마지막 역도 포함해주기
    station_to_lines[stations[-1]].append(line_key)

# 4) 환승/분기 간선 추가 - 동일한 역 이름에 대해서.
for station, lines in station_to_lines.items():
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            u = (lines[i], station)
            v = (lines[j], station)
            graph[u].append((v, 2))
            graph[v].append((u, 2))

# 5) (테스트) '1인천' 분기에서 '신도림' 확인
print("1인천 분기 신도림 이웃 노드:")
for nbr, w in graph[('1인천','신도림')]:
    print(f"  {nbr} — {w}분")
# --- 샘플 노드 이웃 출력 (호선별 첫·중간·마지막 역) ---
print("=== 호선별 샘플 노드 이웃 ===")
for line_key, stations in subway_lines.items():
    mid = len(stations) // 2
    samples = [0, mid, len(stations)-1]
    print(f"\n[line: {line_key}]")
    for idx in samples:
        node = (line_key, stations[idx])
        neigh = graph.get(node, [])
        snippet = neigh[:3]
        more = "..." if len(neigh) > 3 else ""
        print(f"  {node} -> {snippet}{more}")




1인천 분기 신도림 이웃 노드:
=== 호선별 샘플 노드 이웃 ===

[line: 1]
  ('1', '소요산') -> [(('1', '동두천'), 2)]
  ('1', '신길') -> [(('1', '대방'), 2), (('1', '영등포'), 2), (('5하남검단산', '신길'), 2)]
  ('1', '신창') -> [(('1', '온양온천'), 2)]

[line: 1인천]
  ('1인천', '구로') -> [(('1인천', '구일'), 2), (('1', '구로'), 2)]
  ('1인천', '부개') -> [(('1인천', '송내'), 2), (('1인천', '부평'), 2)]
  ('1인천', '인천') -> [(('1인천', '동인천'), 2)]

[line: 1서동탄]
  ('1서동탄', '병점') -> [(('1서동탄', '서동탄'), 2), (('1', '병점'), 2)]
  ('1서동탄', '서동탄') -> [(('1서동탄', '병점'), 2)]
  ('1서동탄', '서동탄') -> [(('1서동탄', '병점'), 2)]

[line: 1광명]
  ('1광명', '금천구청') -> [(('1광명', '광명'), 2), (('1', '금천구청'), 2)]
  ('1광명', '광명') -> [(('1광명', '금천구청'), 2)]
  ('1광명', '광명') -> [(('1광명', '금천구청'), 2)]

[line: 2]
  ('2', '시청') -> [(('2', '을지로입구'), 2), (('2', '충정로'), 2), (('1', '시청'), 2)]...
  ('2', '교대') -> [(('2', '강남'), 2), (('2', '서초'), 2), (('3', '교대'), 2)]
  ('2', '시청') -> [(('2', '을지로입구'), 2), (('2', '충정로'), 2), (('1', '시청'), 2)]...

[line: 2신설동]
  ('2신설동', '성수') -> [(('2신설동', '용답'), 2), (('2', 

In [2]:
import heapq  # 우선순위 큐 구현을 위한 라이브러리

def dijkstra_verbose(start_station, end_station):
    # 시작역과 도착역은 여러 노선에 걸쳐 있을 수 있음
    # 예: '홍대입구'역이 2호선, 경의중앙선 등 여러 노선에 걸쳐 있을 수 있기 때문
    # 그래서 가능한 모든 (노선, 역) 쌍을 리스트로 생성
    start_nodes = [(line, start_station) for line in station_to_lines[start_station]]
    end_nodes = [(line, end_station) for line in station_to_lines[end_station]]

    # 최단 거리 저장용 딕셔너리
    # key: (노선, 역) 튜플, value: 출발지부터 해당 노드까지의 최소 누적 시간
    dist = {}

    # 경로 추적용 딕셔너리
    # key: 현재 노드, value: 현재 노드로 오기 직전 노드
    prev = {}

    # 우선순위 큐 초기화 (힙 구조)
    # (누적시간, 현재 노드) 튜플로 저장
    pq = []

    # 시작 노드들의 거리를 0으로 초기화 후 우선순위 큐에 삽입
    for node in start_nodes:
        heapq.heappush(pq, (0, node))
        dist[node] = 0
        prev[node] = None

    # 다익스트라 알고리즘 수행
    while pq:
        # 누적 시간이 가장 작은 노드부터 꺼냄
        cur_dist, cur_node = heapq.heappop(pq)

        # 만약 현재 노드가 도착 노드 리스트에 있으면 최단 경로 탐색 완료
        if cur_node in end_nodes:
            # 도착 노드부터 출발 노드까지 경로 역추적
            path = []
            while cur_node:
                path.append(cur_node)
                cur_node = prev[cur_node]
            path.reverse()  # 역추적이므로 뒤집어줌
            print_verbose_path(path)  # 경로를 보기 좋게 출력하는 함수 호출
            return path

        # 현재 노드의 인접 노드(이웃)들에 대해 거리 업데이트 시도
        for neighbor, weight in graph[cur_node]:
            new_dist = cur_dist + weight  # 현재까지 누적시간 + 이동시간
            # 만약 neighbor를 아직 방문 안 했거나 새로 계산한 거리가 더 짧으면 갱신
            if neighbor not in dist or new_dist < dist[neighbor]:
                dist[neighbor] = new_dist
                prev[neighbor] = cur_node  # 최단 경로 추적용 기록
                heapq.heappush(pq, (new_dist, neighbor))  # 큐에 새 거리와 노드 삽입

    # 큐가 빌 때까지 도착 노드를 못 찾으면 경로가 없다는 뜻
    print("경로를 찾을 수 없습니다.")
    return None

def print_verbose_path(path):
    print("\n🛤️ 최단 경로:")  # 최단 경로 출력 시작 알림

    total_time = 0  # 총 소요 시간 초기화

    # 첫 번째 노드 정보 저장 (노선, 역)
    last_line, last_station = path[0]

    # 출발역 출력
    print(f"  출발 ▶️ [{last_line}호선] {last_station}")

    # 경로의 두 번째 노드부터 마지막 노드까지 순회
    for i in range(1, len(path)):
        cur_line, cur_station = path[i]        # 현재 노드 (노선, 역)
        prev_line, prev_station = path[i-1]    # 이전 노드 (노선, 역)

        if cur_line != prev_line:
            # 노선이 다르면 환승한 것임
            # 환승 시간은 2분으로 가정
            print(f"  🔁 환승: {prev_line}호선 → {cur_line}호선 @ {cur_station} (2분 소요)")
            total_time += 2  # 환승 시간 누적
        else:
            # 노선이 같으면 같은 노선 내 이동
            # 역 간 이동 시간도 2분으로 가정
            print(f"  ➡ 이동: {prev_station} → {cur_station} ({cur_line}호선, 2분 소요)")
            total_time += 2  # 이동 시간 누적

    # 모든 이동과 환승이 끝난 후 총 소요 시간 출력
    print(f"\n⏱️ 총 소요 시간: {total_time}분\n")



다익스트라 알고리즘이란?
그래프에서 한 정점(노드)에서 다른 모든 정점까지의 최단 거리를 구하는 알고리즘.

음수 가중치가 없는 그래프에서 사용 가능.

우선순위 큐(힙)을 사용해 가장 효율적으로 구현.

핵심 아이디어
출발점에서 시작해 방문하지 않은 노드 중 출발점으로부터 가장 가까운 노드를 하나씩 선택해서 거리를 확정.

선택된 노드에 인접한 노드들의 거리를 갱신.

이 과정을 모든 노드의 거리가 확정될 때까지 반복.

알고리즘 단계
출발 노드의 거리를 0으로 초기화하고, 나머지 노드의 거리는 무한대로 초기화.

방문하지 않은 노드 중 최단 거리가 가장 짧은 노드를 선택.

선택한 노드를 방문 처리하고, 인접 노드의 거리 정보를 업데이트.

2~3 단계를 반복.

모든 노드가 방문 처리되거나, 도착 노드의 거리가 확정되면 종료.

코드 설명
dist: 현재까지 알려진 출발지부터 각 노드까지의 최단 거리

prev: 경로 추적용으로 각 노드의 이전 노드를 저장

pq: 우선순위 큐로, (현재까지의 누적 거리, 노드)를 저장해 가장 가까운 노드부터 처리할 수 있도록 함

start_nodes, end_nodes: 역은 여러 노선에 걸쳐 있을 수 있기 때문에, 노선별로 모든 시작점과 도착점 노드를 만든다.


우선순위 큐에 중복된 노드가 들어갈 수 있음. 이때 큐에서 꺼낸 거리가 현재 저장된 최단 거리보다 크면 무시.

최단 경로를 역추적하는 방법은?

각 노드에 대해 이전 노드를 저장하는 prev 딕셔너리를 활용해, 도착 노드부터 출발 노드까지 역순으로 따라가면서 경로를 만듦.

노드가 (노선, 역) 형태인 이유.

지하철 같은 다중 노선 그래프에서는 동일한 역이라도 다른 노선에 있을 수 있으므로, 각 노드를 명확하게 구분하기 위해 노선과 역을 묶어서 표현합니다.

+ 노선 기입 없이 역만으로 구성하는 것에 대해서 더 생각해보기. 발표를 잘하면 되지 않을까? 기능 추가 하나하나가 가산점이니까.

In [3]:

dijkstra_verbose("서울역", "잠실")


🛤️ 최단 경로:
  출발 ▶️ [4호선] 서울역
  ➡ 이동: 서울역 → 회현 (4호선, 2분 소요)
  ➡ 이동: 회현 → 명동 (4호선, 2분 소요)
  ➡ 이동: 명동 → 충무로 (4호선, 2분 소요)
  ➡ 이동: 충무로 → 동대문역사문화공원 (4호선, 2분 소요)
  🔁 환승: 4호선 → 2호선 @ 동대문역사문화공원 (2분 소요)
  ➡ 이동: 동대문역사문화공원 → 신당 (2호선, 2분 소요)
  ➡ 이동: 신당 → 상왕십리 (2호선, 2분 소요)
  ➡ 이동: 상왕십리 → 왕십리 (2호선, 2분 소요)
  ➡ 이동: 왕십리 → 한양대 (2호선, 2분 소요)
  ➡ 이동: 한양대 → 뚝섬 (2호선, 2분 소요)
  ➡ 이동: 뚝섬 → 성수 (2호선, 2분 소요)
  ➡ 이동: 성수 → 건대입구 (2호선, 2분 소요)
  ➡ 이동: 건대입구 → 구의 (2호선, 2분 소요)
  ➡ 이동: 구의 → 강변 (2호선, 2분 소요)
  ➡ 이동: 강변 → 잠실나루 (2호선, 2분 소요)
  ➡ 이동: 잠실나루 → 잠실 (2호선, 2분 소요)

⏱️ 총 소요 시간: 32분



[('4', '서울역'),
 ('4', '회현'),
 ('4', '명동'),
 ('4', '충무로'),
 ('4', '동대문역사문화공원'),
 ('2', '동대문역사문화공원'),
 ('2', '신당'),
 ('2', '상왕십리'),
 ('2', '왕십리'),
 ('2', '한양대'),
 ('2', '뚝섬'),
 ('2', '성수'),
 ('2', '건대입구'),
 ('2', '구의'),
 ('2', '강변'),
 ('2', '잠실나루'),
 ('2', '잠실')]

In [4]:
import time

start_time = time.time()
dijkstra_verbose("석계", "수원")
end_time = time.time()
print(f"실행 시간: {end_time - start_time}초")


🛤️ 최단 경로:
  출발 ▶️ [1호선] 석계
  ➡ 이동: 석계 → 신이문 (1호선, 2분 소요)
  ➡ 이동: 신이문 → 외대앞 (1호선, 2분 소요)
  ➡ 이동: 외대앞 → 회기 (1호선, 2분 소요)
  ➡ 이동: 회기 → 청량리 (1호선, 2분 소요)
  ➡ 이동: 청량리 → 제기동 (1호선, 2분 소요)
  ➡ 이동: 제기동 → 신설동 (1호선, 2분 소요)
  ➡ 이동: 신설동 → 동묘앞 (1호선, 2분 소요)
  ➡ 이동: 동묘앞 → 동대문 (1호선, 2분 소요)
  ➡ 이동: 동대문 → 종로5가 (1호선, 2분 소요)
  ➡ 이동: 종로5가 → 종로3가 (1호선, 2분 소요)
  ➡ 이동: 종로3가 → 종각 (1호선, 2분 소요)
  ➡ 이동: 종각 → 시청 (1호선, 2분 소요)
  ➡ 이동: 시청 → 서울역 (1호선, 2분 소요)
  ➡ 이동: 서울역 → 남영 (1호선, 2분 소요)
  ➡ 이동: 남영 → 용산 (1호선, 2분 소요)
  ➡ 이동: 용산 → 노량진 (1호선, 2분 소요)
  ➡ 이동: 노량진 → 대방 (1호선, 2분 소요)
  ➡ 이동: 대방 → 신길 (1호선, 2분 소요)
  ➡ 이동: 신길 → 영등포 (1호선, 2분 소요)
  ➡ 이동: 영등포 → 신도림 (1호선, 2분 소요)
  ➡ 이동: 신도림 → 구로 (1호선, 2분 소요)
  ➡ 이동: 구로 → 가산디지털단지 (1호선, 2분 소요)
  ➡ 이동: 가산디지털단지 → 독산 (1호선, 2분 소요)
  ➡ 이동: 독산 → 금천구청 (1호선, 2분 소요)
  ➡ 이동: 금천구청 → 석수 (1호선, 2분 소요)
  ➡ 이동: 석수 → 관악 (1호선, 2분 소요)
  ➡ 이동: 관악 → 안양 (1호선, 2분 소요)
  ➡ 이동: 안양 → 명학 (1호선, 2분 소요)
  ➡ 이동: 명학 → 금정 (1호선, 2분 소요)
  ➡ 이동: 금정 → 군포 (1호선, 2분 소요)
  ➡ 이동: 군포 → 당정 (1호선, 2분 소요)
  ➡ 이동: 당정 → 의왕 (1호선, 2분 소요)


In [12]:
dijkstra_verbose("마곡", "오금")


🛤️ 최단 경로:
  출발 ▶️ [5마천호선] 마곡
  ➡ 이동: 마곡 → 발산 (5마천호선, 2분 소요)
  ➡ 이동: 발산 → 우장산 (5마천호선, 2분 소요)
  ➡ 이동: 우장산 → 화곡 (5마천호선, 2분 소요)
  ➡ 이동: 화곡 → 까치산 (5마천호선, 2분 소요)
  🔁 환승: 5마천호선 → 2까치산호선 @ 까치산 (2분 소요)
  ➡ 이동: 까치산 → 신정네거리 (2까치산호선, 2분 소요)
  ➡ 이동: 신정네거리 → 양천구청 (2까치산호선, 2분 소요)
  ➡ 이동: 양천구청 → 도림천 (2까치산호선, 2분 소요)
  ➡ 이동: 도림천 → 신도림 (2까치산호선, 2분 소요)
  🔁 환승: 2까치산호선 → 2호선 @ 신도림 (2분 소요)
  ➡ 이동: 신도림 → 대림 (2호선, 2분 소요)
  ➡ 이동: 대림 → 구로디지털단지 (2호선, 2분 소요)
  ➡ 이동: 구로디지털단지 → 신대방 (2호선, 2분 소요)
  ➡ 이동: 신대방 → 신림 (2호선, 2분 소요)
  ➡ 이동: 신림 → 봉천 (2호선, 2분 소요)
  ➡ 이동: 봉천 → 서울대입구 (2호선, 2분 소요)
  ➡ 이동: 서울대입구 → 낙성대 (2호선, 2분 소요)
  ➡ 이동: 낙성대 → 사당 (2호선, 2분 소요)
  ➡ 이동: 사당 → 방배 (2호선, 2분 소요)
  ➡ 이동: 방배 → 서초 (2호선, 2분 소요)
  ➡ 이동: 서초 → 교대 (2호선, 2분 소요)
  🔁 환승: 2호선 → 3호선 @ 교대 (2분 소요)
  ➡ 이동: 교대 → 남부터미널 (3호선, 2분 소요)
  ➡ 이동: 남부터미널 → 양재 (3호선, 2분 소요)
  ➡ 이동: 양재 → 매봉 (3호선, 2분 소요)
  ➡ 이동: 매봉 → 도곡 (3호선, 2분 소요)
  ➡ 이동: 도곡 → 대치 (3호선, 2분 소요)
  ➡ 이동: 대치 → 학여울 (3호선, 2분 소요)
  ➡ 이동: 학여울 → 대청 (3호선, 2분 소요)
  ➡ 이동: 대청 → 일원 (3호선, 2분 소요)
  ➡ 이동: 일원 → 수서 (3

[('5마천', '마곡'),
 ('5마천', '발산'),
 ('5마천', '우장산'),
 ('5마천', '화곡'),
 ('5마천', '까치산'),
 ('2까치산', '까치산'),
 ('2까치산', '신정네거리'),
 ('2까치산', '양천구청'),
 ('2까치산', '도림천'),
 ('2까치산', '신도림'),
 ('2', '신도림'),
 ('2', '대림'),
 ('2', '구로디지털단지'),
 ('2', '신대방'),
 ('2', '신림'),
 ('2', '봉천'),
 ('2', '서울대입구'),
 ('2', '낙성대'),
 ('2', '사당'),
 ('2', '방배'),
 ('2', '서초'),
 ('2', '교대'),
 ('3', '교대'),
 ('3', '남부터미널'),
 ('3', '양재'),
 ('3', '매봉'),
 ('3', '도곡'),
 ('3', '대치'),
 ('3', '학여울'),
 ('3', '대청'),
 ('3', '일원'),
 ('3', '수서'),
 ('3', '가락시장'),
 ('3', '경찰병원'),
 ('3', '오금')]